In [4]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

### ESTA FUNCION ES PARA REDUCIR AL 5%. PASASELA SOLO A LOS ARCHIVOS QUE FALTA DESCARGAR. LOS QUE ESTAN SUBIDOS A CLOUD STORAGE YA ESTAN AL 5%.

In [2]:
def parquet_a_parquet_en_carpeta(carpeta_origen, carpeta_destino):
    # Obtener la lista de archivos Parquet en la carpeta de origen
    archivos_parquet = [archivo for archivo in os.listdir(carpeta_origen) if archivo.endswith('.parquet')]

    # Iterar sobre cada archivo Parquet
    for archivo in archivos_parquet:
        ruta_parquet = os.path.join(carpeta_origen, archivo)

        # Leer el archivo Parquet y convertirlo a DataFrame
        tabla_parquet = pq.read_table(ruta_parquet)
        dataframe = tabla_parquet.to_pandas()

        # Realizar las operaciones necesarias en el DataFrame
        dataframe = dataframe.sample(frac=0.05, random_state=123).reset_index(drop=True)

        # Crear la ruta para el archivo Parquet de salida en la carpeta destino
        ruta_parquet_destino = os.path.join(carpeta_destino, archivo.replace('.parquet', '.parquet'))

        # Guardar el DataFrame como archivo Parquet
        tabla_parquet_destino = pd.DataFrame(dataframe)
        tabla_parquet_destino.to_parquet(ruta_parquet_destino, engine='pyarrow')


In [5]:
carpeta_origen = 'uber_descarga_fallos'
carpeta_destino = 'uber_datasets_fallos_con_ETL'

parquet_a_parquet_en_carpeta(carpeta_origen, carpeta_destino)

### ANTES DE HACER LAS TRANSFORMACIONES HAY QUE UNIRLO AL DATASET DE TAXI ZONE

In [6]:
import os
import pandas as pd
import pyarrow.parquet as pq

def realizar_join(carpeta_datos_procesados, carpeta_destino):
    # Inicializa la lista de DataFrames
    dataframes = []

    # Iterar sobre cada archivo Parquet
    for archivo in os.listdir(carpeta_datos_procesados):
        # Verifica si el archivo es de tipo Parquet
        if archivo.endswith('.parquet'):
            # Construye la ruta completa al archivo
            ruta_parquet = os.path.join(carpeta_datos_procesados, archivo)

            # Leer el archivo Parquet y convertirlo a DataFrame
            tabla_parquet = pq.read_table(ruta_parquet)
            datos_procesados = tabla_parquet.to_pandas()

            # Convertir el contenido CSV a un DataFrame de pandas
            taxis_zones = pd.read_csv('Data/taxi+_zone_lookup.csv')

            # Realizar ambos joins en una sola operación
            datos_procesados = pd.merge(
                datos_procesados, taxis_zones,
                left_on='PULocationID', right_on='LocationID', how='inner'
            ).rename(columns={'Borough': 'PUBorough', 'Zone': 'PUZone'}).drop('LocationID', axis=1)

            datos_procesados = pd.merge(
                datos_procesados, taxis_zones,
                left_on='DOLocationID', right_on='LocationID', how='inner'
            ).rename(columns={'Borough': 'DOBorough', 'Zone': 'DOZone'}).drop('LocationID', axis=1)

            # Convertir las columnas a cadena
            datos_procesados['PUBorough'] = datos_procesados['PUBorough'].astype(str)
            datos_procesados['DOBorough'] = datos_procesados['DOBorough'].astype(str)
            datos_procesados['PUZone'] = datos_procesados['PUZone'].astype(str)
            datos_procesados['DOZone'] = datos_procesados['DOZone'].astype(str)

            print(f"Join realizado exitosamente para el archivo: {archivo}")
            dataframes.append(datos_procesados)

            # Crear la ruta para el archivo Parquet de salida en la carpeta destino
            nombre_archivo_final = f'{os.path.splitext(archivo)[0]}_joined.parquet'
            ruta_parquet_destino = os.path.join(carpeta_destino, nombre_archivo_final)

            # Guardar el DataFrame como archivo Parquet
            df_final = pd.concat(dataframes, ignore_index=True)
            df_final.to_parquet(ruta_parquet_destino, engine='pyarrow')

    print(f"Todos los joins y guardados exitosos en {ruta_parquet_destino}.")
    return ruta_parquet_destino




In [7]:
# Uso de la función
carpeta_datos_procesados = 'uber_datasets_fallos_reducidos'
carpeta_destino = 'uber_datasets_join'
realizar_join(carpeta_datos_procesados, carpeta_destino)

Join realizado exitosamente para el archivo: fhvhv_tripdata_2023-05.parquet
Join realizado exitosamente para el archivo: fhvhv_tripdata_2023-06.parquet
Join realizado exitosamente para el archivo: fhvhv_tripdata_2023-07.parquet
Join realizado exitosamente para el archivo: fhvhv_tripdata_2023-08.parquet
Join realizado exitosamente para el archivo: fhvhv_tripdata_2023-09.parquet
Join realizado exitosamente para el archivo: fhvhv_tripdata_2023-10.parquet
Join realizado exitosamente para el archivo: fhvhv_tripdata_2023-11.parquet
Join realizado exitosamente para el archivo: fhvhv_tripdata_2023-12.parquet
Todos los joins y guardados exitosos en uber_datasets_join\fhvhv_tripdata_2023-12_joined.parquet.


'uber_datasets_join\\fhvhv_tripdata_2023-12_joined.parquet'

### A LOS ARCHIVOS REDUCIDOS SE LOS PASA POR LA SIGUIENTE FUNCION QUE ES LA DE TRANSFORMACIÓN. UNA VEZ TRANSFORMADOS LOS ARCHIVOS REDUCIDOS, HAY QUE DESCARGAR LOS ARCHIVOS DE CLOUD STORAGE QUE YA ESTAN AL 5% Y ESTAN TRANSFORMADOS Y GUARDARLOS EN LA MISMA CARPETA DONDE VAYAN A GUARDARSE LOS QUE PASES POR LA SIGUIENTE FUNCIÓN.

In [16]:
def procesar_y_guardar_datos_uber(carpeta_origen, carpeta_destino):
    # Inicializa la lista de DataFrames
    dataframes = []
    
    # Extrae el nombre de la carpeta
    nombre_carpeta = os.path.basename(os.path.normpath(carpeta_origen))

    # Itera sobre cada archivo Parquet en la carpeta de origen
    for archivo in os.listdir(carpeta_origen):
        # Verifica si el archivo es de tipo Parquet
        if archivo.endswith('.parquet'):
            # Construye la ruta completa al archivo
            ruta_completa = os.path.join(carpeta_origen, archivo)

            # Lee el archivo Parquet y almacena el DataFrame en la lista
            table = pq.read_table(ruta_completa)
            uber = table.to_pandas()

            # Aplica las transformaciones necesarias al DataFrame
            uber = uber[uber['hvfhs_license_num'] == 'HV0003']
            uber.fillna(0.00, inplace=True)
            
            columnas_a_sumar = ['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'driver_pay']
            uber['total_fare'] = uber[columnas_a_sumar].sum(axis=1)

            columnas_a_borrar = ['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'driver_pay']
            uber = uber.drop(columnas_a_borrar, axis=1)

            columnas_a_convertir = ['pickup_datetime', 'dropoff_datetime']
            for columna in columnas_a_convertir:
                uber[columna] = pd.to_datetime(uber[columna], errors='coerce')

            orden = ['pickup_datetime', 'PULocationID', 'PUBorough', 'PUZone', 'dropoff_datetime', 'DOLocationID', 'DOBorough', 'DOZone', 'trip_miles', 'trip_time', 'tips', 'total_fare']
            uber = uber[orden]

            uber.rename(columns={'pickup_datetime': 'PUDatetime', 'dropoff_datetime': 'DODatetime'}, inplace=True)

            uber['PUDate'] = uber['PUDatetime'].dt.date
            uber['PUTime'] = uber['PUDatetime'].dt.time

            uber['DODate'] = uber['DODatetime'].dt.date
            uber['DOTime'] = uber['DODatetime'].dt.time

            columnas_a_borrar = ['PUDatetime', 'DODatetime']
            uber = uber.drop(columnas_a_borrar, axis=1)

            orden = ['PUDate', 'PUTime', 'PULocationID', 'PUBorough', 'PUZone', 'DODate', 'DOTime', 'DOLocationID', 'DOBorough', 'DOZone', 'trip_miles', 'trip_time', 'tips', 'total_fare']
            uber = uber[orden]

            # Almacena el DataFrame procesado directamente en la lista
            dataframes.append(uber)

    # Combina todos los DataFrames en uno solo
    df_final = pd.concat(dataframes, ignore_index=True)

    # Construye el nombre del archivo Parquet final
    nombre_archivo_final = f'{nombre_carpeta}_merged.parquet'

    # Construye la ruta completa al archivo Parquet de destino
    ruta_parquet_destino = os.path.join(carpeta_destino, nombre_archivo_final)
    
    # Guarda el DataFrame como archivo Parquet---------------------------
    #table_final = pq.Table.from_pandas(df_final)
    #pq.write_table(table_final, ruta_parquet_destino)

    # Guardar el DataFrame como archivo Parquet-----------------------------
    df_final = pd.concat(dataframes, ignore_index=True)
    df_final.to_parquet(ruta_parquet_destino, engine='pyarrow')


    # Opcional: Devuelve la ruta del archivo Parquet final si es necesario
    return ruta_parquet_destino


In [18]:
carpeta_origen = 'uber_datasets_join'
carpeta_destino = 'uber_datasets_transformados'

procesar_y_guardar_datos_uber(carpeta_origen, carpeta_destino)

'uber_datasets_transformados\\uber_datasets_join_merged.parquet'

In [35]:
import os
import pandas as pd
import pyarrow.parquet as pq
import numpy as np

def procesar_y_guardar_datos_yellow(carpeta_datos_procesados, carpeta_destino):
    # Inicializa la lista de DataFrames
    dataframes = []

    # Iterar sobre cada archivo Parquet en la carpeta de origen
    for archivo in os.listdir(carpeta_datos_procesados):
        # Verifica si el archivo es de tipo Parquet
        if archivo.endswith('.parquet'):
            # Construye la ruta completa al archivo
            ruta_parquet = os.path.join(carpeta_datos_procesados, archivo)

            # Leer el archivo Parquet y convertirlo a DataFrame
            tabla_parquet = pq.read_table(ruta_parquet)
            yellow_taxis = tabla_parquet.to_pandas()

            # Conversion de las columnas en tipo de dato str a fecha y hora
            yellow_taxis["tpep_pickup_datetime"] = pd.to_datetime(yellow_taxis["tpep_pickup_datetime"], format='%Y-%m-%d %H:%M:%S')
            yellow_taxis["tpep_dropoff_datetime"] = pd.to_datetime(yellow_taxis["tpep_dropoff_datetime"], format='%Y-%m-%d %H:%M:%S')

            # Creacion de Columna nueva
            yellow_taxis["trip_datatime"] = yellow_taxis["tpep_dropoff_datetime"] - yellow_taxis["tpep_pickup_datetime"]

            # Filtrados los campos donde el año sea menor a 2019 y eliminar esas filas del DataFrame
            yellow_taxis = yellow_taxis[yellow_taxis["tpep_pickup_datetime"].dt.year >= 2019]

            # Filtrados los campos donde el año sea menor a 2019 y eliminar esas filas del DataFrame
            yellow_taxis = yellow_taxis[yellow_taxis["tpep_dropoff_datetime"].dt.year >= 2019]

            # Elimincion de filas con valores nulos en la columna 'passenger_count'
            yellow_taxis = yellow_taxis.dropna(subset=['passenger_count'])

            # Conversion los 0s a NaNs en la columna 'passenger_count'
            yellow_taxis.loc[yellow_taxis['passenger_count'] == 0, 'passenger_count'] = np.nan

            # Interpolar valores nulos en la columna 'passenger_count'
            yellow_taxis.loc[:, 'passenger_count'] = yellow_taxis['passenger_count'].interpolate(method='linear')

            # Convertir la columna 'passenger_count' a tipo entero
            yellow_taxis['passenger_count'] = yellow_taxis['passenger_count'].astype(int)

            # Dropeo de columnas de poca utilidad
            yellow_taxis.drop('store_and_fwd_flag', axis=1, inplace=True)
            yellow_taxis.drop('VendorID', axis=1, inplace=True)

            # Filtrado
            filtro_total_amount_mayores = yellow_taxis['total_amount'] > 10000

            # Eliminacion de las filas con valores mayor a 100000
            yellow_taxis = yellow_taxis.drop(yellow_taxis[filtro_total_amount_mayores].index)

            # Eliminacion de las filas que tienen total amount negativos
            yellow_taxis = yellow_taxis.drop(yellow_taxis[yellow_taxis['total_amount'] < 0].index)

            ## SUMA DE TAZAS Y CREACION DE COLUMNA 'TOTAL_FARE' ##

            # Se crea una lista con las columnas a sumar
            columnas_a_sumar = ['fare_amount', 'extra', 'mta_tax','tolls_amount','improvement_surcharge','congestion_surcharge']

            # Se crea una nueva columna 'total_fare' que contiene la suma de las columnas de costos
            yellow_taxis['total_fare'] = yellow_taxis[columnas_a_sumar].sum(axis=1)

            # Se crea una lista con las columnas a borrar
            columnasAborrar = ['fare_amount', 'extra', 'mta_tax','tolls_amount','improvement_surcharge','congestion_surcharge', 'total_amount']

            # Se dropean las columnas enlistadas
            yellow_taxis = yellow_taxis.drop(columnasAborrar, axis=1)

            ## FIN SUMA DE TAZAS Y CREACION DE COLUMNA 'TOTAL_FARE' ##

            # Extraccion del componente de tiempo en segundos y convertirlo a entero
            yellow_taxis['trip_datatime'] = pd.to_timedelta(yellow_taxis['trip_datatime']) 
            yellow_taxis['trip_time'] = yellow_taxis['trip_datatime'].dt.total_seconds().astype(int)

            # Renombrado de columnas
            yellow_taxis.rename(columns={'tip_amount': 'tips', 'trip_distance':'trip_miles'}, inplace=True)

            ## CONVERSION A DATETIME Y SEPARACION POR COLUMNAS ##

            # Lista de nombres de columnas que deseas convertir a tipo datetime
            columnas_a_convertir = ['tpep_pickup_datetime','tpep_dropoff_datetime']

            # Itera sobre cada columna y realiza la conversión a tipo datetime
            for columna in columnas_a_convertir:
                yellow_taxis[columna] = pd.to_datetime(yellow_taxis[columna], errors='coerce')

            # Crea dos nuevas columnas: 'fecha' y 'hora' para recogida
            yellow_taxis['PUDate'] = yellow_taxis['tpep_pickup_datetime'].dt.date
            yellow_taxis['PUTime'] = yellow_taxis['tpep_pickup_datetime'].dt.time

            # Crea dos nuevas columnas: 'fecha' y 'hora' para arribo
            yellow_taxis['DODate'] = yellow_taxis['tpep_dropoff_datetime'].dt.date
            yellow_taxis['DOTime'] = yellow_taxis['tpep_dropoff_datetime'].dt.time

            # Lista de columnas datetime a eliminar
            columnasAborrar = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']

            # Eliminacion de columnas datetime
            yellow_taxis.drop(columnasAborrar, axis=1, inplace=True)

            # Ordenado de columnas
            orden = yellow_taxis[['PUDate','PUTime','PULocationID','PUBorough','PUZone','DODate','DOTime','DOLocationID','DOBorough','DOZone','trip_miles','trip_time','tips','passenger_count','payment_type','total_fare']]

            # Almacena el DataFrame procesado directamente en la lista
            dataframes.append(orden)

            print(f"Join realizado exitosamente para el archivo: {archivo}")

            # Crear la ruta para el archivo Parquet de salida en la carpeta destino
            nombre_archivo_final = f'{os.path.splitext(archivo)[0]}_joined.parquet'
            ruta_parquet_destino = os.path.join(carpeta_destino, nombre_archivo_final)

            # Guardar el DataFrame como archivo Parquet
            df_final = pd.concat(dataframes, ignore_index=True)
            df_final.to_parquet(ruta_parquet_destino, engine='pyarrow')

            print(f"Join y guardado exitosos para el archivo: {archivo}")

    print(f"Todos los joins y guardados exitosos en {carpeta_destino}.")
    return ruta_parquet_destino



In [36]:
carpeta_origen = 'yellow_join'
carpeta_destino = 'yellow_taxis_transformado'

procesar_y_guardar_datos_yellow(carpeta_origen, carpeta_destino)

Join realizado exitosamente para el archivo: yellow_tripdata_2023-05_joined.parquet
Join y guardado exitosos para el archivo: yellow_tripdata_2023-05_joined.parquet
Join realizado exitosamente para el archivo: yellow_tripdata_2023-07_joined.parquet
Join y guardado exitosos para el archivo: yellow_tripdata_2023-07_joined.parquet
Join realizado exitosamente para el archivo: yellow_tripdata_2023-08_joined.parquet
Join y guardado exitosos para el archivo: yellow_tripdata_2023-08_joined.parquet
Join realizado exitosamente para el archivo: yellow_tripdata_2023-09_joined.parquet
Join y guardado exitosos para el archivo: yellow_tripdata_2023-09_joined.parquet
Join realizado exitosamente para el archivo: yellow_tripdata_2023-10_joined.parquet
Join y guardado exitosos para el archivo: yellow_tripdata_2023-10_joined.parquet
Join realizado exitosamente para el archivo: yellow_tripdata_2023-11_joined.parquet
Join y guardado exitosos para el archivo: yellow_tripdata_2023-11_joined.parquet
Todos los 

'yellow_taxis_transformado\\yellow_tripdata_2023-11_joined_joined.parquet'

### UNA VEZ QUE SE TIENEN TODOS LOS PARQUET DE UBER DESCARGADOS SE PASA LA SIGUIENTE FUNCION

In [5]:
def concatenar_parquet_en_carpeta(carpeta):
    # Inicializa la lista de DataFrames
    dataframes = []

    # Extrae el nombre de la carpeta
    nombre_carpeta = os.path.basename(os.path.normpath(carpeta))

    # Itera sobre cada archivo en la carpeta
    for archivo in os.listdir(carpeta):
        # Verifica si el archivo es de tipo Parquet
        if archivo.endswith('.parquet'):
            # Construye la ruta completa al archivo
            ruta_completa = os.path.join(carpeta, archivo)

            # Lee el archivo Parquet y almacena el DataFrame en la lista
            table = pq.read_table(ruta_completa)
            df = table.to_pandas()

            # Almacena el DataFrame directamente en la lista
            dataframes.append(df)

    # Combina todos los DataFrames en uno solo
    df_final = pd.concat(dataframes, ignore_index=True)

    # Convierte el DataFrame a una tabla PyArrow
    table_final = pa.Table.from_pandas(df_final)

    # Construye el nombre del archivo Parquet con la última parte del nombre de la carpeta
    ruta_parquet_final = f'{nombre_carpeta}_merged.parquet'

    # Guarda la tabla en formato Parquet
    pq.write_table(table_final, ruta_parquet_final)

    return ruta_parquet_final

In [6]:
carpeta_parquet = 'uber_datasets_transformados'
resultado_parquet = concatenar_parquet_en_carpeta(carpeta_parquet)

### ESTE CODIGO LO EJECUTAS PARA QUE TE DESCARGUE TODOS LOS ARCHIVOS DE UBER DESDE GOOGLE CLOUD. LE VA A TOMAR UN CACHITO DESCARGAR, PERO ES MEJOR A DESCARGAR ARCHIVO POR ARCHIVO.

In [ ]:
!gsutil -m cp \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-02.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-03.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-04.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-05.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-06.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-07.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-08.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-09.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-10.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-11.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2019-12.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-01.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-02.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-03.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-04.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-05.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-06.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-07.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-08.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-09.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-10.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-11.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2020-12.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-01.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-02.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-03.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-04.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-05.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-06.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-07.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-08.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-09.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-10.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-11.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2021-12.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-01.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-02.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-03.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-04.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-05.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-06.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-07.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-08.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-09.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-10.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-11.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2022-12.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2023-01.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2023-02.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2023-04.parquet" \
"gs://proyectofinal_nyc/uber/fhvhv_tripdata_2023-06.parquet" \
"yellow_descargado/" # este "yellow_descargado/" es la ruta relativa de donde quiero que me los guarde. vos pone la que te sirva a vos

## ESTE ES EL CODIGO PARA DESCARGAR YELLOW, NO LE DES PELOTA

In [41]:
!gsutil -m cp \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-01.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-02.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-03.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-04.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-05.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-06.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-07.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-08.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-09.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-10.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-11.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-12.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-01.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-02.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-03.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-04.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-05.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-06.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-07.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-08.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-09.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-10.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-11.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2020-12.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-01.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-02.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-03.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-04.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-05.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-06.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-07.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-08.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-09.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-10.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-11.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2021-12.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-01.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-02.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-03.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-04.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-05.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-06.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-07.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-08.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-09.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-10.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-11.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2022-12.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2023-01.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2023-02.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2023-03.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2023-04.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2023-06.parquet" \
"gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2023-12.parquet" \
"yellow_descargado/"

Copying gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-01.parquet...
/ [0 files][    0.0 B/105.3 MiB]                                                
Copying gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-02.parquet...
/ [0 files][    0.0 B/203.9 MiB]                                                
Copying gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-03.parquet...
/ [0 files][    0.0 B/314.5 MiB]                                                
Copying gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-04.parquet...
/ [0 files][    0.0 B/419.6 MiB]                                                
-
Copying gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-05.parquet...
- [0 files][    0.0 B/525.9 MiB]                                                
Copying gs://proyectofinal_nyc/yellow_taxis/yellow_tripdata_2019-06.parquet...
- [0 files][    0.0 B/624.0 MiB]                                                
Copying gs://proyectofinal_nyc/yellow_